## imports

In [3]:
from src.load import load_full_network_from_df
from src.util import AggregateFullTransferNetwork

import matplotlib.pyplot as plt

from collections import defaultdict

# Path studies
Do the following studies of paths 

* Path lengths of players that 
    1. End at a club
    2. Go through a club
* Player values as they go through clubs

## Rankings
get the rankings for certain years to see how they correlate with path profiles